<a href="https://colab.research.google.com/github/MarcusLongton/bank_marketing_analysis/blob/main/bank_marketing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import os

import time

In [2]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')
df_fast = pd.read_csv('bank-additional.csv', sep =';')

In [3]:
df_fast.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
# Smaller faster df for training slow models lime SVC
df_fast.shape

(4119, 21)

In [6]:
df.shape

(41188, 21)

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [43]:
# Categorical Feature: Bar Plots

categorical_features = ['job', 'marital', 'education', 'poutcome']

for feature in categorical_features:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=feature, hue='y')
    plt.title(f'Subscription Rate by {feature.capitalize()}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'images/{feature}_barplot.png', bbox_inches='tight')
    plt.close()


# Numerical Features: Histograms & Boxplots

numerical_features = ['age', 'campaign', 'previous', 'euribor3m']

for feature in numerical_features:
    # Histogram
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=feature, hue='y', kde=True, bins=20)
    plt.title(f'Distribution of {feature.capitalize()}')
    plt.savefig(f'images/{feature}_histogram.png')
    plt.close()

    # Boxplot
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x='y', y=feature)
    plt.title(f'Subscription vs. {feature.capitalize()}')
    plt.savefig(f'images/{feature}_boxplot.png')
    plt.close()

# Correlation Matrix

plt.figure(figsize=(12, 8))
corr = df.select_dtypes(include=['int64', 'float64']).corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.savefig('images/correlation_matrix.png', bbox_inches='tight')
plt.close()


# Classification Report Visualization

# Example results_df (modify with your actual data)
results_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'SVM', 'Decision Tree', 'Dummy Classifier'],
    'Test F1': [0.80, 0.75, 0.70, 0.45],
    'Test Precision': [0.62, 0.58, 0.55, 0.35],
    'Test Recall': [0.61, 0.59, 0.53, 0.35],
    'Training Time': [16.4, 45.7, 0.8, 0.1]
})

# Melt for seaborn plotting
melted_df = results_df.melt(id_vars='Model', var_name='Metric', value_name='Score')

plt.figure(figsize=(12, 6))
sns.barplot(data=melted_df, x='Model', y='Score', hue='Metric')
plt.title('Model Performance Comparison')
plt.ylim(0, 1)
plt.legend(loc='upper right')
plt.savefig('images/model_performance.png', bbox_inches='tight')
plt.close()

### Many of our categorical features will need to go through feature engineering. Ordinal and One Hot Encoding to be specific.

In [7]:
# Check for any missing values
df.isna().value_counts()

,,,,,,,,,,,,,,,,,,,,,count
age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,
False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,41188


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

## Business Understanding
### What are the factors that are consequential in predicting if the client will subscribe to a new term deposit. Moreover, what is the best classification method that will give us the most useful predictive power?

- To answer this we will looking at several different Classification models (**K-Nearest Neighbors, Logsitic Regression, Decision Trees, Support Vector Machines**) to determine which is best suited for this specific use case.


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [9]:
# Firslty will drop 'duration' column from both dataframes. The documentation tells us it should be "discarded if the intention is to have a realistic predictive model."
df = df.drop(columns=['duration'], axis=1)
df_fast = df_fast.drop(columns=['duration'], axis=1)

In [10]:
# Separate our Features (X) and our Target (y)
X = df.drop(columns=['y'])
y = df['y']

In [11]:
# Separate our Features (X) and our Targe (y) ** for the smaller (faster) dataset
# This is to improve the speed of our GridSearch.
X_fast = df_fast.drop(columns=['y'], axis=1)
y_fast = df_fast['y']

In [12]:
# Create a label encoder
le_fast = LabelEncoder()
y_fast = le_fast.fit_transform(y_fast)

# Sort through categorical and numerical features
categorical_features_fast = X_fast.select_dtypes(include=['object']).columns.tolist()
numerical_features_fast = X_fast.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocess the data by scaling and one hot encoding
preprocessor_fast = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_fast),
        ('cat', OneHotEncoder(drop='first', handle_unknown = 'ignore'), categorical_features_fast)
    ]
)

In [13]:
# Create a label encoder
le = LabelEncoder()
y = le.fit_transform(y)

# Sort through categorical and numerical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocess the data by scaling and one hot encoding
preprocessor_full = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [14]:
# Split our fast Data
X_train_fast, X_test_fast, y_train_fast, y_test_fast = train_test_split(X_fast, y_fast, test_size=0.2, random_state=42)

In [15]:
# Split our fully sized DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Apply our preprocessing after splitting our data.
X_train_fast_processed = preprocessor_fast.fit_transform(X_train_fast)
X_test_fast_processed = preprocessor_fast.fit_transform(X_test_fast)

X_train_processed = preprocessor_full.fit_transform(X_train)
X_test_processed = preprocessor_full.fit_transform(X_test)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [17]:
# Creating a Dummy Classifier to have as a baseline.
dummy_clf = DummyClassifier(strategy='stratified', random_state=42)

In [18]:
# Fit the model and predict for our (df_fast)
dummy_clf.fit(X_train_fast_processed, y_train_fast)

# Create predictions
y_pred_fast_dummy = dummy_clf.predict(X_test_fast_processed)

In [19]:
# Same thing for the full df
dummy_clf.fit(X_train_processed, y_train)

# Create predictions
y_pred_full_dummy = dummy_clf.predict(X_test_processed)

In [20]:
# Evaluate our baseline performance
print("Fast Dataset Baseline Performance:")
print(classification_report(y_test_fast, y_pred_fast_dummy))
print(f"ROC-AUC: {roc_auc_score(y_test_fast, dummy_clf.predict_proba(X_test_fast_processed)[:, 1]):.3f}\n")

# For the FULL dataset
print("Full Dataset Baseline Performance:")
print(classification_report(y_test, y_pred_full_dummy))
print(f"ROC-AUC: {roc_auc_score(y_test, dummy_clf.predict_proba(X_test_processed)[:, 1]):.3f}")

Fast Dataset Baseline Performance:
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       732
           1       0.05      0.05      0.05        92

    accuracy                           0.78       824
   macro avg       0.46      0.46      0.46       824
weighted avg       0.79      0.78      0.78       824

ROC-AUC: 0.465

Full Dataset Baseline Performance:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      7303
           1       0.11      0.10      0.10       935

    accuracy                           0.80      8238
   macro avg       0.50      0.50      0.50      8238
weighted avg       0.80      0.80      0.80      8238

ROC-AUC: 0.496


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [21]:
# Update OneHotEncoder to handle unseen categories
preprocessor_full = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)

# Reprocess data
X_train_processed = preprocessor_full.fit_transform(X_train)
X_test_processed = preprocessor_full.transform(X_test)

# Now retrain the model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_processed, y_train)
y_pred_log_reg = log_reg.predict(X_test_processed)

### Problem 9: Score the Model

What is the accuracy of your model?

In [22]:
# Scoring Metrics for the Logistic Regression
print("Logistic Regression (Full Dataset):")
print(classification_report(y_test, y_pred_log_reg))
print(f"ROC-AUC: {roc_auc_score(y_test, log_reg.predict_proba(X_test_processed)[:, 1]):.3f}")

# For the FULL dataset
print("Full Dataset Baseline Performance:")
print(classification_report(y_test, y_pred_full_dummy))
print(f"ROC-AUC: {roc_auc_score(y_test, dummy_clf.predict_proba(X_test_processed)[:, 1]):.3f}")

Logistic Regression (Full Dataset):
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      7303
           1       0.64      0.21      0.32       935

    accuracy                           0.90      8238
   macro avg       0.77      0.60      0.63      8238
weighted avg       0.88      0.90      0.87      8238

ROC-AUC: 0.780
Full Dataset Baseline Performance:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      7303
           1       0.11      0.10      0.10       935

    accuracy                           0.80      8238
   macro avg       0.50      0.50      0.50      8238
weighted avg       0.80      0.80      0.80      8238

ROC-AUC: 0.496


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [32]:
# Create a dictionary of the models that we would like to compare for this evaluation

models = {
    'k_nearest_neighbors': (KNeighborsClassifier()),
    'logistic_regression': (LogisticRegression(max_iter=1000, random_state=42)),
    'decision_tree': (DecisionTreeClassifier(random_state=42)),
    'svm': (SVC(random_state=42)),
    'dummy': DummyClassifier(strategy='stratified', random_state=42)
}

In [34]:
# List to store results
results = []

# Loop through models
for model_name, model in models.items():
    # Track training time
    start_time = time.time()

    # Fit the model
    model.fit(X_train_processed, y_train)
    train_time = time.time() - start_time

    # Get predictions and scores
    y_train_pred = model.predict(X_train_processed)
    y_test_pred = model.predict(X_test_processed)

    # Calculate accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    precicison = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    # Store results
    results.append({
        'Model': model_name,
        'Train Time': round(train_time, 3),
        'Train Accuracy': round(train_acc, 4),
        'Test Accuracy': round(test_acc, 4),
        'Precision': round(precicison, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4)
    })

# Create comparison dataframe
results_df = pd.DataFrame(results).sort_values(by='Test Accuracy', ascending=False)

# Display results
print(results_df)

                 Model  Train Time  Train Accuracy  Test Accuracy  Precision  \
3                  svm     112.728          0.9054         0.8971     0.6160   
1  logistic_regression       0.231          0.9014         0.8969     0.6387   
0  k_nearest_neighbors       0.004          0.9146         0.8923     0.5465   
2        decision_tree       1.353          0.9954         0.8370     0.3027   
4                dummy       0.001          0.8006         0.7995     0.1056   

   Recall  F1 Score  
3  0.2471    0.3527  
1  0.2118    0.3181  
0  0.3016    0.3887  
2  0.3348    0.3179  
4  0.1027    0.1041  


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [26]:
'''Our top three models (SVM, Logisitc Regression, and K-Nearest Neighbors) are signigicantly better than Decision Trees.
We will focus on Hyper-parameter tuning of these three models on the smaller DataFrame first, and then take the top one and perform
and extensive tuning process on the full size DataFrame.'''

'Our top three models (SVM, Logisitc Regression, and K-Nearest Neighbors) are signigicantly better than Decision Trees.\nWe will focus on Hyper-parameter tuning of these three models on the smaller DataFrame first, and then take the top one and perform \nand extensive tuning process on the full size DataFrame.'

Here it is important to make the observation that as we perform our GridSeach, we have changed the defaul scoring metric from 'accuracy' to 'f1'. F1 score is more acceptable for our use case. It balances precision and recall and also performs well for imbalanced datasets.

In [35]:
# Dictionary for our three best models
best_models = {
    'k_nearest_neighbors': (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7, 9, 15, 20]}),
    'logistic_regression': (LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42), {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}),
    'svm': (SVC(random_state=42), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 0.001, 0.1, 1]}),
    'decision_tree': (DecisionTreeClassifier(random_state=42), {'max_depth': [5, 10, 13, 15]})
}


# Store results
results = []

# Loop through models and perform GridSearch on FAST dataset
for model_name, (model, params) in best_models.items():
    # Initialize GridSearchCV
    grid = GridSearchCV(
        estimator=model,
        param_grid=params,
        cv=5,  # 5-fold cross-validation
        scoring='f1',
        n_jobs=-1  # Use all CPU cores
    )

    # Fit on FAST dataset
    start_time = time.time()
    grid.fit(X_train_fast_processed, y_train_fast)
    train_time = time.time() - start_time

    # Get best model and retrain on FULL dataset
    best_model = grid.best_estimator_
    best_model.fit(X_train_processed, y_train)  # Use full data for final training

    # Evaluate
    y_train_pred = best_model.predict(X_train_processed)
    y_test_pred = best_model.predict(X_test_processed)

    # Save results
    results.append({
        'Model': model_name,
        'Best Params': grid.best_params_,
        'Train Time (Fast)': round(train_time, 3),
        'Train Accuracy (Full)': round(accuracy_score(y_train, y_train_pred), 4),
        'Test Accuracy (Full)': round(accuracy_score(y_test, y_test_pred), 4),
        'Precision': round(precision_score(y_test, y_test_pred), 4),
        'Recall': round(recall_score(y_test, y_test_pred), 4),
        'F1 Score': round(f1_score(y_test, y_test_pred), 4)
    })

# Create DataFrame
results_df = pd.DataFrame(results)
print(results_df)

                 Model                                       Best Params  \
0  k_nearest_neighbors                               {'n_neighbors': 20}   
1  logistic_regression                 {'C': 0.1, 'solver': 'liblinear'}   
2                  svm  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}   
3        decision_tree                                  {'max_depth': 5}   

   Train Time (Fast)  Train Accuracy (Full)  Test Accuracy (Full)  Precision  \
0              2.807                 0.9036                0.8948     0.5983   
1              1.028                 0.8313                0.8264     0.3486   
2             22.412                 0.8982                0.8948     0.6090   
3              0.339                 0.9045                0.8962     0.6124   

   Recall  F1 Score  
0  0.2214    0.3232  
1  0.6096    0.4436  
2  0.2032    0.3047  
3  0.2332    0.3377  


In [36]:
# Define hyperparameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'class_weight': [None, 'balanced'],    # Handle class imbalance
    'solver': ['liblinear', 'lbfgs']       # Solvers for optimization
}

# Initialize model and GridSearch
logreg = LogisticRegression(max_iter=1000, random_state=42)
grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    scoring='f1',  # Prioritize F1-score
    cv=5,
    n_jobs=-1
)

# Track training time
start_time = time.time()

# Fit grid search on FULL dataset
grid_search.fit(X_train_processed, y_train)

# Calculate training time
train_time = time.time() - start_time

# Get best model
best_logreg = grid_search.best_estimator_

# Generate predictions
y_train_pred = best_logreg.predict(X_train_processed)
y_test_pred = best_logreg.predict(X_test_processed)

# Calculate metrics
results = {
    'Model': ['Tuned Logistic Regression (Full)'],
    'Best Parameters': [grid_search.best_params_],
    'Train Time (s)': [round(train_time, 3)],
    'Train Accuracy': [round(accuracy_score(y_train, y_train_pred), 4)],
    'Test Accuracy': [round(accuracy_score(y_test, y_test_pred), 4)],
    'Precision': [round(precision_score(y_test, y_test_pred), 4)],
    'Recall': [round(recall_score(y_test, y_test_pred), 4)],
    'F1-Score': [round(f1_score(y_test, y_test_pred), 4)]
}

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

                              Model  \
0  Tuned Logistic Regression (Full)   

                                     Best Parameters  Train Time (s)  \
0  {'C': 10, 'class_weight': 'balanced', 'solver'...          16.436   

   Train Accuracy  Test Accuracy  Precision  Recall  F1-Score  
0          0.8329         0.8281     0.3515  0.6086    0.4456  


In [38]:
# Create pipeline with feature selection, SMOTE, and logistic regression
pipeline = make_pipeline(
    SMOTE(random_state=42),  # Apply SMOTE first
    SelectKBest(score_func=f_classif),  # Feature selection after resampling
    LogisticRegression(random_state=42, max_iter=1000)
)

# Define parameter grid
param_grid = {
    'selectkbest__k': [15, 20, 25, 'all'],  # Number of features to keep
    'logisticregression__C': [0.1, 1, 10, 100],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__class_weight': [None, 'balanced', {0:1, 1:3}],
    'logisticregression__solver': ['liblinear']  # Only solver that works with L1/L2
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit on training data (use original processed data)
grid_search.fit(X_train_processed, y_train)

# Get best model
best_model = grid_search.best_estimator_

# Generate predictions
y_train_pred = best_model.predict(X_train_processed)
y_test_pred = best_model.predict(X_test_processed)

# Get feature names (if using ColumnTransformer)
feature_names = preprocessor_full.get_feature_names_out()
selected_features = feature_names[best_model.named_steps['selectkbest'].get_support()]

# Create results table
results = {
    'Model': ['Optimized Logistic Regression'],
    'Best Params': [grid_search.best_params_],
    'Selected Features': [selected_features.tolist()],
    'Train F1': [f1_score(y_train, y_train_pred)],
    'Test F1': [f1_score(y_test, y_test_pred)],
    'Test Precision': [precision_score(y_test, y_test_pred)],
    'Test Recall': [recall_score(y_test, y_test_pred)]
}

results_df = pd.DataFrame(results)
print(results_df)
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))
print("\nSelected Features:", selected_features)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
                           Model  \
0  Optimized Logistic Regression   

                                         Best Params  \
0  {'logisticregression__C': 0.1, 'logisticregres...   

                                   Selected Features  Train F1  Test F1  \
0  [num__campaign, num__pdays, num__previous, num...  0.452429    0.437   

   Test Precision  Test Recall  
0        0.339243     0.613904  

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.85      0.89      7303
           1       0.34      0.61      0.44       935

    accuracy                           0.82      8238
   macro avg       0.64      0.73      0.67      8238
weighted avg       0.88      0.82      0.84      8238


Selected Features: ['num__campaign' 'num__pdays' 'num__previous' 'num__emp.var.rate'
 'num__cons.price.idx' 'num__cons.conf.idx' 'num__euribor3m'
 'num__nr.employed' 'cat__job_blue-co

In [39]:
results_df

,Model,Best Params,Selected Features,Train F1,Test F1,Test Precision,Test Recall
0,Optimized Logistic Regression,"{'logisticregression__C': 0.1, 'logisticregres...","[num__campaign, num__pdays, num__previous, num...",0.452429,0.437,0.339243,0.613904


##### Questions